In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker, cm
import seaborn as sns
from ipywidgets import *
import math

#绘图元素比例  比较小
sns.set_context('paper', font_scale=2)
#设置显示中文字体
font = {'family' : 'SimHei',
#         'weight' : 'bold',
        'size'   : '15'}
plt.rc('font', **font)               # 步骤一（设置字体的更多属性）
plt.rc('axes', unicode_minus=False)

In [2]:
def f_2d(x1, x2):
    '''优化的目标函数 有2个变量'''
    return 0.5 * x1 ** 2 + 2 * x2 ** 2

def f_grad(x1, x2):
    '''x1和x2的梯度'''
    dfdx1 =  x1
    dfdx2 = 4 * x2
    return dfdx1, dfdx2

def train_2d(trainer, lr,epoch=50,init_x1=-4,init_x2=-4):
    """自定义训练器:trainer
       lr:学习率
       epoch:轮次
       init_x1:x1的初始值 便于观察
       init_x2:x1的初始值
    """
    x1, x2 = init_x1, init_x2
    s_x1, s_x2 = 0, 0
    res = [(x1, x2)]
    for i in range(epoch):
        x1, x2, s_x1, s_x2, lr = trainer(x1, x2, s_x1, s_x2, lr)
        res.append((x1, x2))
    return res

def plot_2d(res, figsize=(10, 8), title=None):
    x1_, x2_ = zip(*res)
    fig = plt.figure(figsize=figsize)
    plt.plot([0], [0], 'r*', ms=15)
    plt.text(0.0, 0.25, 'min', color='k')
    plt.plot(x1_[0], x2_[0], 'ro', ms=10)
    plt.text(x1_[0]+0.2, x2_[0]-0.15, 'start', color='k')
    plt.plot(x1_, x2_, '-o', color='r')
    
    plt.plot(x1_[-1], x2_[-1], 'bo',ms=10)
    plt.text(x1_[-1]+0.1, x2_[-1]-0.4, 'end', color='k')
    
    x1 = np.linspace(-5,5, 100)
    x2 = np.linspace(-5,5, 100)
    x1, x2 = np.meshgrid(x1, x2)
    #画出等高线区域，并用彩虹颜色填充，设置透明度
    cp=plt.contourf(x1, x2, f_2d(x1, x2),alpha=0.75, cmap=cm.rainbow)
    #画出等高线
    C=plt.contour(x1, x2, f_2d(x1, x2),colors='black')
    plt.clabel(C,inline=True,fontsize=15)
    cbar = fig.colorbar(cp)
    cbar.set_label('Loss')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title(title)
    plt.show()
    x1,x2=res[-1]
    loss=f_2d(x1,x2)
    print('最小损失值:',loss,'x1:',x1,' x2:',x2)

In [3]:
def sgd(x1, x2, s1, s2, lr):
    dfdx1, dfdx2 = f_grad(x1, x2)
    return (x1 - lr * dfdx1, x2 - lr * dfdx2, 0, 0, lr)

#定义了4个变量控件，可以随时调节，查看效果 (最小值，最大值，步长)
@interact(lr=(0, 1, 0.001),epoch=(0,100,1),init_x1=(-5,5,0.1),init_x2=(-5,5,0.1),continuous_update=False)
def visualize_gradient_descent(lr=0.05,epoch=50,init_x1=-4,init_x2=-2.4):
    res = train_2d(sgd, lr,epoch,init_x1,init_x2)
    plot_2d(res,(12,8), title='SGD')
# 0.4 11 -4.00 -2.40

interactive(children=(FloatSlider(value=0.05, description='lr', max=1.0, step=0.001), IntSlider(value=50, desc…

最小损失值: 0.04736423610933763 x1: -0.30777990110685327  x2: -3.425394462494306e-05


In [4]:
@interact(lr=(0, 0.99, 0.001), gamma=(0, 0.99, 0.001),continuous_update=False,
         epoch=(0,20,1),init_x1=(-5,5,0.1),init_x2=(-5,5,0.1))
def visualize_sgd_momentum(lr=0.1, gamma=0.1,epoch=10,init_x1=-4,init_x2=-2.4):
    '''lr: learning rate
    gamma: parameter for momentum sgd
    每次都会根据上一次的动量来进行更新
    '''
    
    def momentum(x1, x2, v1, v2, lr):
        dfdx1, dfdx2 = f_grad(x1, x2)
        v1 = gamma * v1 + lr * dfdx1
        v2 = gamma * v2 + lr * dfdx2
        x1 = x1 - v1
        x2 = x2 - v2
        return (x1, x2, v1, v2, lr)
    
    res = train_2d(momentum, lr,epoch,init_x1,init_x2)
    plot_2d(res, title='SGD with Momentum')
# 0.40， 0.08， 10， -4.00， -2.40

interactive(children=(FloatSlider(value=0.1, description='lr', max=0.99, step=0.001), FloatSlider(value=0.1, d…

In [5]:
@interact(lr=(0, 0.99, 0.01), gamma=(0, 0.50, 0.01),
          continuous_update=False,epoch=(0,10,1),init_x1=(-5,5,0.1),init_x2=(-5,5,0.1))
def visualize_sgd_inertia(lr=0.1, gamma=0.1,epoch=10,init_x1=-4,init_x2=-2.4):
    '''lr: learning rate
    gamma: parameter for inertia sgd'''
    
    def inertia(x1, x2, v1, v2, lr):
        dfdx1, dfdx2 = f_grad(x1, x2)
        v1 = gamma * v1 + (1 - gamma) * dfdx1
        v2 = gamma * v2 + (1 - gamma) * dfdx2
        x1 = x1 - lr * v1
        x2 = x2 - lr * v2
        return (x1, x2, v1, v2, lr)
    
    res = train_2d(inertia, lr,epoch,init_x1,init_x2)
    plot_2d(res, title='inertia')
# 0.41， 0.10， 10， -4.00， -2.40

interactive(children=(FloatSlider(value=0.1, description='lr', max=0.99, step=0.01), FloatSlider(value=0.1, de…

In [6]:
@interact(lr=(0, 1.5, 0.01),
          continuous_update=False,epoch=(0,10,1),init_x1=(-5,5,0.1),init_x2=(-5,5,0.1))
def visualize_adagrad(lr=0.1,epoch=10,init_x1=-4,init_x2=-2.4):
    '''lr: learning rate'''
    def adagrad_2d(x1, x2, s1, s2, lr):
        g1, g2 = f_grad(x1, x2)
        eps = 1e-6
        s1 += g1 ** 2
        s2 += g2 ** 2
        x1 -= lr / math.sqrt(s1 + eps) * g1
        x2 -= lr / math.sqrt(s2 + eps) * g2
        return x1, x2, s1, s2, lr
    
    res = train_2d(adagrad_2d, lr,epoch,init_x1,init_x2)
    plot_2d(res, title='Adagrad')
# 1.25， 10， -4.00， -2.40

interactive(children=(FloatSlider(value=0.1, description='lr', max=1.5, step=0.01), IntSlider(value=10, descri…

In [7]:
@interact(lr=(0, 1, 0.001), 
          gamma=(0, 0.99, 0.001),
          continuous_update=False,epoch=(0,15,1),init_x1=(-5,5,0.1),init_x2=(-5,5,0.1))
def visualize_rmsprop(lr=0.1, gamma=0.9,epoch=50,init_x1=-4,init_x2=-4):
    '''lr: learning rate, 
       gamma: momentum'''  
    def rmsprop_2d(x1, x2, s1, s2, lr):
        eps = 1e-6
        g1, g2 = f_grad(x1, x2)
        s1 = gamma * s1 + (1 - gamma) * g1 ** 2
        s2 = gamma * s2 + (1 - gamma) * g2 ** 2
        x1 -= lr / math.sqrt(s1 + eps) * g1
        x2 -= lr / math.sqrt(s2 + eps) * g2
        return x1, x2, s1, s2, lr

    res = train_2d(rmsprop_2d, lr,epoch,init_x1,init_x2)
    plot_2d(res, title='RMSProp')
# 0.67， 0.25， 9， -4.00， -2.40

interactive(children=(FloatSlider(value=0.1, description='lr', max=1.0, step=0.001), FloatSlider(value=0.9, de…

In [8]:
@interact(lr=(0, 1, 0.001), 
          beta1=(0, 0.999, 0.001),
          beta2=(0, 0.999, 0.001),
          continuous_update=False,epoch=(0,100,1),init_x1=(-5,5,0.1),init_x2=(-5,5,0.1))
def visualize_adam(lr=0.1, beta1=0.9, beta2=0.999,epoch=10,init_x1=-4,init_x2=-2.4):
    '''lr: learning rate
    beta1: parameter for E(g)
    beta2: parameter for E(g^2)
    '''  
    def Deltax(m, n, g, t):
        eps = 1.0E-6
        m = beta1 * m + (1 - beta1) * g
        n = beta2 * n + (1 - beta2) * g*g
        m_hat = m / (1 - beta1**t)
        n_hat = n / (1 - beta2**t)
        dx = lr * m_hat / (math.sqrt(n_hat) + eps)
        return m, n, dx
        
    def adam_2d(x1, x2, m1, n1, m2, n2, lr, t):
        '''m1, m2: E(g1), E(g2)
           n1, n2: E(g1^2), E(g2^2) where E() is expectation
           lr: learning rate
           t: time step'''
        eps = 1e-6
        g1, g2 = f_grad(x1, x2)
        m1, n1, dx1 = Deltax(m1, n1, g1, t)
        m2, n2, dx2 = Deltax(m2, n2, g2, t)       
        x1 -= dx1
        x2 -= dx2
        return x1, x2, m1, n1, m2, n2, lr
    
    def train_adam(trainer, lr,epoch=10,init_x1=-4,init_x2=-4):
        """Train a 2d object function with a customized trainer"""
        x1, x2 = init_x1,init_x2
        m1, n1, m2, n2 = 0, 0, 0, 0
        res = [(x1, x2)]
        for i in range(epoch):
            x1, x2, m1, n1, m2, n2, lr = trainer(x1, x2, m1, n1, m2, n2, lr, i+1)
            res.append((x1, x2))
        return res
    
    res = train_adam(adam_2d, lr,epoch,init_x1,init_x2)
    plot_2d(res, title='adam')
# 0.6 0.24 10 -4 -2.4

interactive(children=(FloatSlider(value=0.1, description='lr', max=1.0, step=0.001), FloatSlider(value=0.9, de…